In [ ]:
import os
import sqlite3
from multiprocessing import freeze_support
from modin.db_conn import ModinDatabaseConnection
import modin.pandas as mpd
%load_ext autoreload
%autoreload 2
os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
def load_data_from_db(con):
    try:
        df = mpd.read_sql("SELECT * FROM data", con)
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        raise


freeze_support()
dbfile = '/home/tompouce/workspaces/mafat-challenge/train_data_for_competition/mini_training_set.db'

conn = ModinDatabaseConnection('sqlalchemy', f'sqlite:///{dbfile}')

# Can use get_connection to get underlying sqlalchemy engine
conn.get_connection()
db_df = load_data_from_db(conn)
print(db_df.head())

In [ ]:
# "23-04 to 18-05"
db_df["Datetime"] = mpd.to_datetime(db_df["Datetime"])
db_df["Datetime"]
# db_df.groupby("Device_ID").apply(lambda x: (x-x["Datetime"].min()).dt.days)

In [ ]:
def get_domain_counts(db_df, pivot=False):
    domain_counts = db_df.groupby(["Device_ID","Domain_Name","Target"]).count()
    domain_counts = domain_counts.reset_index()
    domain_counts = domain_counts[["Device_ID","Domain_Name","Target","Datetime"]]
    domain_counts.rename(columns={"Datetime":"count"}, inplace=True)
    if pivot:
        pivot_matrix = train_domain_counts.pivot(index='source', columns='target', values='count').fillna(0)
        return pivot_matrix
    return domain_counts


In [ ]:
def get_train_test_masks(domain_counts, test_size=0.2, random_state=42):
    # Get unique device IDs and their corresponding targets
    device_target_df = domain_counts.groupby('Device_ID')['Target'].first().reset_index()
    
    # Perform stratified split on device IDs
    train_device_ids, test_device_ids = train_test_split(
        device_target_df['Device_ID'],
        test_size=test_size,
        random_state=random_state,
        stratify=device_target_df['Target']
    )
    
    # Create mask for train/test split in domain_counts
    train_mask = domain_counts['Device_ID'].isin(train_device_ids)
    test_mask = domain_counts['Device_ID'].isin(test_device_ids)
    
    # Print statistics
    print(f"Total devices: {len(device_target_df)}")
    print(f"Train devices: {len(train_device_ids)}")
    print(f"Test devices: {len(test_device_ids)}")
    print(f"\nTrain samples: {len(domain_counts[train_mask])}")
    print(f"Test samples: {len(domain_counts[test_mask])}")
    
    # Print class distribution
    print("\nTarget distribution in train set:")
    print(domain_counts[train_mask].groupby('Target').size() / len(domain_counts[train_mask]))
    print("\nTarget distribution in test set:")
    print(domain_counts[test_mask].groupby('Target').size() / len(domain_counts[test_mask]))
    
    return train_mask, test_mask


In [ ]:
print("keep in mind that the best resolution will be achieved with a resolution of urls/chain of urls, not domains")
print("Cluster url walks")
print("I want to cluster urls/url walks from a given domain, to 3 categories")
print("positive correlation, zero correlation, negative correlation, to_label")

In [ ]:
import numpy as np
import ray

def compute_chunked_covariance(pivot_matrix, batch_size=2000):
    # Convert to numpy array for faster computation
    matrix_dense = pivot_matrix.to_numpy()
    matrix_centered = matrix_dense - np.mean(matrix_dense, axis=0)

    # Initialize parameters
    n_cols = pivot_matrix.shape[1]
    futures = []

    # Submit tasks to Ray
    for i in range(0, n_cols, batch_size):
        batch_end = min(i + batch_size, n_cols)
        futures.append(calculate_chunk_covariance.remote(matrix_centered, i, batch_end, pivot_matrix.columns))

    # Collect results and combine
    cov_chunks = []
    for future in ray.get(futures):
        start_idx, end_idx, chunk_cov = future
        chunk_df = mpd.DataFrame(
            chunk_cov,
            index=pivot_matrix.columns[start_idx:end_idx],
            columns=pivot_matrix.columns
        )
        cov_chunks.append(chunk_df)

    # Combine all chunks
    return mpd.concat(cov_chunks)

In [ ]:
def melt_covariance_matrix(covariance_matrix):
    # Reset index to make it a column
    melted = covariance_matrix.reset_index()
    
    # Melt the dataframe
    melted = melted.melt(
        id_vars=['index'],
        var_name='target',
        value_name='covariance'
    )
    
    # Rename the 'index' column to 'source'
    melted = melted.rename(columns={'index': 'source'})
    
    # Remove duplicate pairs (e.g., if A->B exists, remove B->A)
    melted = melted[melted['source'] < melted['target']]
    
    # Remove rows where source equals target
    melted = melted[melted['source'] != melted['target']]
    
    return melted.reset_index(drop=True)

In [ ]:
def filter_and_transform_covariance(melted_covariance, abs_threshold=0.03, log_threshold=10):
    # Create a copy to avoid modifying the original dataframe
    result = melted_covariance
    # Create corrected_cov column
    result['corrected_cov'] = result['covariance'].copy()
    
    # Apply absolute threshold filter
    result.loc[abs(result['corrected_cov']) < abs_threshold, 'corrected_cov'] = 0
    
    # Apply log transformation for values above log_threshold
    high_vals_mask = abs(result['corrected_cov']) > log_threshold
    result.loc[high_vals_mask, 'corrected_cov'] = result.loc[high_vals_mask, 'corrected_cov'].apply(
        lambda x: np.log2(abs(x)) * np.sign(x)
    )
    
    return result

In [ ]:
def get_device_domain_fractions(domain_counts):
    # Calculate total counts per device
    device_totals = domain_counts.groupby('Device_ID')['count'].sum()
    
    # Calculate fractions by dividing each count by the device total
    domain_fractions = domain_counts.copy()
    domain_fractions['fraction'] = domain_fractions.apply(
        lambda row: row['count'] / device_totals[row['Device_ID']], 
        axis=1
    )
    
    return domain_fractions[['Device_ID', 'Domain_Name', 'Target', 'count', 'fraction']]

In [ ]:
def compute_domain_target_correlation(domain_counts):
    # Group by Domain_Name and calculate mean Target and count
    domain_stats = domain_counts.groupby('Domain_Name').agg({
        'Target': 'mean',
        'count': ['mean', 'std', 'count']
    }).reset_index()
    
    # Flatten column names
    domain_stats.columns = ['Domain_Name', 'target_mean', 'count_mean', 'count_std', 'n_devices']
    
    # Calculate correlation coefficient
    # We only include domains that appear in multiple devices for statistical significance
    significant_domains = domain_stats[domain_stats['n_devices'] > 1]
    
    # Calculate correlation and p-value
    correlation = mpd.DataFrame({
        'Domain_Name': significant_domains['Domain_Name'],
        'target_correlation': significant_domains['target_mean'],
        'avg_count': significant_domains['count_mean'],
        'count_std': significant_domains['count_std'],
        'n_devices': significant_domains['n_devices']
    }).sort_values('target_correlation', ascending=False)
    
    return correlation

In [ ]:
# Get domain counts and create pivot matrix
domain_counts = get_domain_counts(db_df)
del db_df
pivot_matrix = domain_counts.pivot(index='Domain_Name', columns='Device_ID', values='count').fillna(0)

# Compute covariance matrix and melt it
covariance_matrix = compute_chunked_covariance(pivot_matrix)
melted_cov = melt_covariance_matrix(covariance_matrix)

# Apply filtering and transformation to covariance
corrected_cov = filter_and_transform_covariance(melted_cov)

# Get device domain fractions from original domain counts
device_fractions = get_device_domain_fractions(domain_counts)
# Compute domain-target correlations
domain_correlations = compute_domain_target_correlation(domain_counts)

print("Shapes:")
print(f"Corrected covariance: {corrected_cov.shape}")
print(f"Device fractions: {device_fractions.shape}")